<a href="https://colab.research.google.com/github/vladgap/Various/blob/main/neural_network_OOP_241021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Perceptron -- neuron with an activation function.
Inputs and their weights are numbered 0 to(n-1), bias is n

In [1]:
import numpy as np

In [2]:
class Perceptron:
    """A single neuron with the sigmoid activation function.
       Attributes:
          inputs: The number of inputs in the perceptron, not counting the bias.
          bias:   The bias term. By defaul it's 1.0."""

    def __init__(self, inputs, bias = 1.0):
        """Return a new Perceptron object with the specified number of inputs (+1 for the bias).""" 
        self.weights = (np.random.rand(inputs+1) * 2) - 1 
        self.bias = bias

    def run(self, x):
        """Run the perceptron. x is a python list with the input values."""
        sum = np.dot(np.append(x,self.bias),self.weights)
        return self.sigmoid(sum)

    def set_weights(self, w_init):
        """Overrides the np.random.rand() weights and the bias weight"""
        # w_init is a list of floats. Organize it as you'd like.
        self.weights=np.array(w_init)

    def sigmoid(self, x):
        # return the output of the sigmoid function applied to x
        return 1/(1+np.exp(-x))

# Testing the perceptron

## AND gate

In [3]:
neuron = Perceptron(inputs=2)
neuron.set_weights([10,10,-15]) #AND gate

print("Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))


Gate:
0 0 = 0.0000003059
0 1 = 0.0066928509
1 0 = 0.0066928509
1 1 = 0.9933071491


## OR gate

In [4]:
neuron = Perceptron(inputs=2)
neuron.set_weights([10,10,-5]) #OR gate

print("Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))


Gate:
0 0 = 0.0066928509
0 1 = 0.9933071491
1 0 = 0.9933071491
1 1 = 0.9999996941


## NAND gate

In [5]:
neuron = Perceptron(inputs=2)
neuron.set_weights([-10,-10,15]) #NAND gate

print("Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))

Gate:
0 0 = 0.9999996941
0 1 = 0.9933071491
1 0 = 0.9933071491
1 1 = 0.0066928509


# Multilayer perceptron

## XOR gate=OR+NAND+AND.


In [17]:
class MultiLayerPerceptron:     
    """A multilayer perceptron class that uses the Perceptron class above.
       Attributes:
          layers:  A python list with the number of elements (incl bias) per layer. Incl the input layer.
          bias:    The bias term. The same bias is used for all neurons.
          eta:     The learning rate."""

    def __init__(self, layers, bias = 1.0):
        """Return a new MLP object with the specified parameters.""" 
        self.layers = np.array(layers,dtype=object)
        self.bias = bias
        self.network = [] # The list of lists of neurons
        self.values = []  # The list of lists of neurons' output values        
        
        # 2 nested loops to create neurons layer by layer
        for i in range(len(self.layers)): # outer loop iterates on each layer
            self.values.append([]) #The new list of values will be filled with zeros, for every neuron in the layer. 
            self.network.append([])
            self.values[i] = [0.0 for j in range(self.layers[i])]
            if i > 0:      #network[0] is the input layer, so it has no neurons
                for j in range(self.layers[i]): # inner loop iterates on each neuron in a layer
                    self.network[i].append(Perceptron(inputs = self.layers[i-1], bias = self.bias))
        
        self.network = np.array([np.array(x) for x in self.network],dtype=object)
        self.values = np.array([np.array(x) for x in self.values],dtype=object)

    def set_weights(self, w_init): # set_weights of the MultiLayer class
        """Set the weights. 
           w_init is a list of lists with the weights for all but the input layer."""
        for i in range(len(w_init)):
            for j in range(len(w_init[i])):
                self.network[i+1][j].set_weights(w_init[i][j]) # set_weights of the Perceptron class

    def printWeights(self):
        print()
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                print("Layer",i+1,"Neuron",j,self.network[i][j].weights)
        print()

    def run(self, x):
        """Feed a sample x into the MultiLayer Perceptron."""
        x = np.array(x,dtype=object)
        self.values[0] = x
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):  
                self.values[i][j] = self.network[i][j].run(self.values[i-1])
        return self.values[-1]

### XOR gate

In [23]:
#test code
mlp = MultiLayerPerceptron(layers=[2,2,1])  #mlp
mlp.set_weights([[[-10,-10,15],[15,15,-10]],[[10,10,-15]]])
mlp.printWeights()
print("MLP:")
print ("0 0 = {0:.10f}".format(mlp.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mlp.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mlp.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mlp.run([1,1])[0]))
print ()
print (mlp.network[1][0].weights) # network adds Perceptron to each layer. It has attribute "weights"


Layer 2 Neuron 0 [-10 -10  15]
Layer 2 Neuron 1 [ 15  15 -10]
Layer 3 Neuron 0 [ 10  10 -15]

MLP:
0 0 = 0.0066958493
0 1 = 0.9923558642
1 0 = 0.9923558642
1 1 = 0.0071528098

[-10 -10  15]
